<a href="https://colab.research.google.com/github/JesseHenson/School/blob/main/csc525_Mod2_ct_opt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import pandas as pd

In [65]:
df = pd.read_csv("https://gist.githubusercontent.com/dhar174/14177e1d874a33bfec565a07875b875a/raw/7aa9afaaacc71aa0e8bc60b38111c24e584c74d8/data.csv", header=0,names=["age","height","weight","gender","game_type"])
df.head()

,age,height,weight,gender,game_type
0,21,60,238,0,Platformer
1,14,78,176,1,Strategy
2,10,67,216,1,Strategy
3,19,69,185,1,RPG
4,34,72,138,0,Platformer


In [66]:
# All features are continuous, so no encoding necessary. 

df.describe()

,age,height,weight,gender
count,200.000000,200.000000,200.000000,200.000000
mean,22.760000,62.890000,176.155000,0.470000
std,9.434908,8.854372,46.126666,0.500352
min,8.000000,48.000000,100.000000,0.000000
25%,14.000000,55.000000,136.750000,0.000000
50%,23.000000,63.000000,180.500000,0.000000
75%,31.000000,70.250000,214.500000,1.000000
max,38.000000,78.000000,250.000000,1.000000


In [67]:
# There are four target classes - multi-nomial classification required 
# KNN is capable of multi-nomial classification
# Use 4 centroids for count
df.game_type.value_counts()

RPG           55
Strategy      54
Platformer    46
Action        45
Name: game_type, dtype: int64

In [68]:
# No imputation or dropping of null values required

df.isna().sum()

age          0
height       0
weight       0
gender       0
game_type    0
dtype: int64

In [97]:
# after manually trying a few types of algorithms, distance metrics and leaf-size
# the configuration below seems to get the best model score.
# I fit the model to the data, 
# which found the euclidian distnace of each point from the centroid and grouped 
# them by which was closest.
# I then used the clusters to classify all of the test data. This was then scored 
# against the test labels that I held aside. Which gave me an accuracy of .25
# I then created a prediction dataframe, along side the actual 
  

from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.model_selection import train_test_split

X = df.drop('game_type', axis=1)
y = df.game_type

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
model = KNeighborsClassifier(n_neighbors=7,leaf_size=100, algorithm="brute", weights='distance',metric="euclidean")
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

results = pd.DataFrame({"predictions":model.predict(X_test), "actual":y_test})
results.head()

0.25


,predictions,actual
95,Action,Action
15,RPG,RPG
30,Platformer,RPG
158,Strategy,Strategy
128,Platformer,Strategy


In [106]:
# tried to improve score by scaling the records. However, I think there are just too few records
# to get a good score

from sklearn.preprocessing import StandardScaler, MinMaxScaler
X_copy = df.drop('game_type', axis=1)
y_copy = df.game_type

std_scaler = StandardScaler()
min_max_scaler = MinMaxScaler()
X_scale = std_scaler.fit_transform(X_copy)
X_scale2 = min_max_scaler.fit_transform(X_copy)

X_train, X_test, y_train, y_test = train_test_split(X_scale,y, test_size=0.2, random_state=42)
model3 = KNeighborsClassifier(n_neighbors=4,leaf_size=200, algorithm="brute", weights='distance',metric="euclidean")
model3.fit(X_train, y_train)
print(f"StandardScaler: {model3.score(X_test, y_test)}")

X_train, X_test, y_train, y_test = train_test_split(X_scale2,y, test_size=0.2, random_state=42)
model4 = KNeighborsClassifier(n_neighbors=4,leaf_size=200, algorithm="brute", weights='distance',metric="euclidean")
model4.fit(X_train, y_train)
print(f'MinMaxScaler: {model4.score(X_test, y_test)}')


StandardScaler: 0.175
MinMaxScaler: 0.15


In [75]:
age = input("enter age: ")
height = input("enter height (in inches): ")
weight = input("enter weight (in lbs): ")
gender = input("enter gender (1 for male, 0 for female): ")

prediction = model.predict([[age, height, weight, gender]])
print(f"We'd recommend {prediction[0]} type games for you")

enter Age20
enter height60
enter weight250
enter gender (1 for male, 0 for female1
We'd recommend RPG type games for you


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:561: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)
